In [ ]:
!pip install openai 
!pip install langchain
!pip install flask 
!pip install langchain_community
!pip install bcrypt

In [ ]:
import os

# Directly set your API key here (be careful not to expose your key in public repositories)
os.environ["OPENAI_API_KEY"] = "your_api_key_here"

In [22]:
import sqlite3

def create_tables():
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()

    # Users table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL
    )""")

    # Sessions table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS sessions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        learning_language TEXT,
        known_language TEXT,
        proficiency_level TEXT,
        current_module TEXT DEFAULT 'Module 1',
        current_submodule TEXT DEFAULT 'Submodule 1',
        start_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users(id)
    )""")

    # Conversations table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS conversations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        session_id INTEGER,
        user_response TEXT,
        tutor_response TEXT,
        performance_score INTEGER DEFAULT 0,
        next_submodule_or_review TEXT,
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (session_id) REFERENCES sessions(id)
    )""")

    # Mistakes table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS mistakes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id INTEGER,
    conversation_id INTEGER,
    module TEXT,
    submodule TEXT,
    error_type TEXT,
    incorrect_response TEXT,
    correct_response TEXT,
    FOREIGN KEY (session_id) REFERENCES sessions(id),
    FOREIGN KEY (conversation_id) REFERENCES conversations(id)
    )""")

    conn.commit()
    conn.close()

# Call this function once to initialize the database
create_tables()


In [31]:
import bcrypt

def signup(username, password):
    """Registers a new user with hashed password."""
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()

    # Hash the password before storing
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

    try:
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        print("Signup successful! You can now log in.")
    except sqlite3.IntegrityError:
        print("Username already exists. Try a different one.")

    conn.close()

def login(username, password):
    """Authenticates user login by verifying hashed password."""
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()

    cursor.execute("SELECT id, password FROM users WHERE username = ?", (username,))
    user = cursor.fetchone()

    conn.close()

    if user and bcrypt.checkpw(password.encode('utf-8'), user[1]):
        print("Login successful!")
        return user[0]  # Return user ID
    else:
        print("Invalid username or password.")
        return None


def get_user_session(user_id):
    """Fetches the latest session details for a user."""
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()

    cursor.execute("""
        SELECT id, learning_language, known_language, proficiency_level, current_module, current_submodule
        FROM sessions WHERE user_id = ? ORDER BY start_time DESC LIMIT 1
    """, (user_id,))

    result = cursor.fetchone()
    conn.close()

    if result:
        return {
            "session_id": result[0],
            "learning_language": result[1],
            "known_language": result[2],
            "proficiency_level": result[3],
            "current_module": result[4],
            "current_submodule": result[5]
        }
    return None  # No active session found

def create_user_session(user_id, learning_language, known_language, proficiency_level):
    """Creates a new learning session for the user."""
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()

    cursor.execute("""
        INSERT INTO sessions (user_id, learning_language, known_language, proficiency_level)
        VALUES (?, ?, ?, ?)
    """, (user_id, learning_language, known_language, proficiency_level))

    conn.commit()
    conn.close()
    print("New session started successfully.")


def save_conversation(user_id, session_id, user_response, tutor_response, performance_score=0, next_submodule_or_review=''):
    """Stores structured conversation data."""
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()
    
    cursor.execute("""
        INSERT INTO conversations (user_id, session_id, user_response, tutor_response, performance_score, next_submodule_or_review)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (user_id, session_id, user_response, tutor_response, performance_score, next_submodule_or_review))

    conn.commit()
    conn.close()


# ---------- Module Progression Functions ----------
def update_user_session_in_db(session_id, new_module, new_submodule):
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()
    cursor.execute("""
        UPDATE sessions 
        SET current_module = ?, current_submodule = ?
        WHERE id = ?
    """, (new_module, new_submodule, session_id))
    conn.commit()
    conn.close()

def update_progress(session, cycle_count, correct_answer_count):
    THRESHOLD_CYCLES = 5
    if cycle_count >= THRESHOLD_CYCLES:
        submodule_num = int(session["current_submodule"].split()[1])
        module_num = int(session["current_module"].split()[1])
        if submodule_num < 3:
            new_submodule = f"Submodule {submodule_num + 1}"
            new_module = session["current_module"]
        else:
            new_module = f"Module {module_num + 1}"
            new_submodule = "Submodule 1"
        session["current_module"] = new_module
        session["current_submodule"] = new_submodule
        update_user_session_in_db(session["session_id"], new_module, new_submodule)
        print(f"Advanced to {new_module} → {new_submodule}")
        cycle_count = 0
        correct_answer_count = 0
    return session, cycle_count, correct_answer_count

# ---------- Revised Summarize Chat History Function ----------
def summarize_chat_history(chat_history_str):
    max_length = 1000
    if len(chat_history_str) > max_length:
        summary_prompt = f"Summarize the following conversation in one concise paragraph:\n{chat_history_str}"
        summary = llm.invoke(summary_prompt)
        return summary
    else:
        return chat_history_str

# ---------- Database Helper for Mistakes ----------
def save_mistake(session_id, conversation_id, error_type, incorrect, correct, module=None, submodule=None):
    conn = sqlite3.connect("chatbot.db")
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO mistakes (session_id, conversation_id, module, submodule, error_type, incorrect_response, correct_response)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (session_id, conversation_id, module, submodule, error_type, incorrect, correct))
    conn.commit()
    conn.close()


In [34]:
# --- User Authentication Flow ---

print("Welcome to the Language Tutor!")
choice = input("Do you want to (1) Sign up or (2) Log in? ")

username = input("Enter username: ")
password = input("Enter password: ")

user_id = None

if choice == "1":
    signup(username, password)
    # After signup, ask the user to log in
    user_id = login(username, password)
elif choice == "2":
    user_id = login(username, password)
else:
    print("Invalid choice. Please restart the program.")

if user_id is None:
    print("Authentication failed. Please restart the program.")
else:
    # Check if the user already has an active session
    session = get_user_session(user_id)
    if session:
        print(f"Welcome back! You're learning {session['learning_language']} with assistance in {session['known_language']}.")
        print(f"Your current progress: {session['current_module']} → {session['current_submodule']}")
    else:
        # If no session exists, it's the first login.
        print("It looks like this is your first session!")
        learning_language = input("Which language do you want to learn? ")
        known_language = input("What is your native language? ")
        proficiency_level = input("What is your proficiency level (beginner/intermediate/advanced)? ")

        create_user_session(user_id, learning_language, known_language, proficiency_level)
        # Retrieve the newly created session details
        session = get_user_session(user_id)
        print(f"Great! You've started learning {learning_language} with assistance in {known_language}.")
        print(f"Your starting progress: {session['current_module']} → {session['current_submodule']}")

# At this point, the user is authenticated and has an active session.

Welcome to the Language Tutor!
Login successful!
Welcome back! You're learning hindi with assistance in english.
Your current progress: Module 1 → Submodule 3


In [35]:
import sqlite3
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage

# ---------- Initialization ----------
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

memory = ConversationBufferMemory(
    return_messages=True, 
    memory_key="chat_history", 
    input_key="user_response"
)

# ---------- Define Prompt Templates for Beginners ----------
teaching_template_beginners = PromptTemplate(
    input_variables=["learning_language", "current_module", "current_submodule", "chat_history"],
    template="""You are a language tutor teaching a beginner student learning {learning_language}.

Current Lesson: **Module:** {current_module} → **Submodule:** {current_submodule}
Note: This lesson is for beginners. Clearly explain the concept and explicitly list the key points the student must remember.

**Teaching Phase:**  
Explain the concept clearly with examples and include written pronunciation guides (e.g., phonetic breakdown).  
Key points to emphasize:
- [Key Point 1]
- [Key Point 2]
- [Key Point 3]

For context, here is a brief summary of previous interactions:
{chat_history}

Please present the lesson without asking any questions.
"""
)

testing_template_beginners = PromptTemplate(
    input_variables=["learning_language", "current_module", "current_submodule", "chat_history"],
    template="""You are a language tutor helping a beginner student learn {learning_language}.

Current Lesson: **Module:** {current_module} → **Submodule:** {current_submodule}

**Testing Phase:**  
Based strictly on the lesson you just presented—including the key points mentioned—ask a simple, direct text-based question that the student can answer using the information provided.  
For context, here is a summary of the lesson so far:
{chat_history}

Please ask your question now.
"""
)

# ---------- Define Prompt Templates for Intermediate/Advanced (Non-Beginners) ----------
teaching_template = PromptTemplate(
    input_variables=["learning_language", "current_module", "current_submodule", "chat_history"],
    template="""You are a language tutor teaching a student learning {learning_language}.

Current Lesson: **Module:** {current_module} → **Submodule:** {current_submodule}

**Teaching Phase:**  
Explain the concept clearly with examples and include written pronunciation guides (e.g., phonetic breakdown).  
For context, here is a brief summary of previous interactions:
{chat_history}

Please present the lesson without asking any questions.
"""
)

testing_template = PromptTemplate(
    input_variables=["learning_language", "current_module", "current_submodule", "chat_history"],
    template="""You are a language tutor helping a student learn {learning_language}.

Current Lesson: **Module:** {current_module} → **Submodule:** {current_submodule}

**Testing Phase:**  
Based on the lesson, ask a clear, text-based question that requires the student to respond using the information provided.
For context, here is a summary of the lesson so far:
{chat_history}

Please ask your question now.
"""
)

# ---------- Evaluation and Mistake Summary Templates (Same for all proficiency levels) ----------
evaluation_template = PromptTemplate(
    input_variables=["learning_language", "current_module", "current_submodule", "user_response", "chat_history"],
    template="""You are a language tutor evaluating a student's written answer in {learning_language}.

Current Lesson: **Module:** {current_module} → **Submodule:** {current_submodule}

**User Response:** {user_response}

**Evaluation Phase:**  
Analyze the student's response. If it is correct, include "Great job" in your feedback and indicate readiness to move on.  
If it is incorrect, specify the mistake (e.g., pronunciation, vocabulary, grammar) and provide the correct answer with an explanation.  
Include a brief context summary:
{chat_history}

Please provide your evaluation.
"""
)

mistake_summary_template = PromptTemplate(
    input_variables=["learning_language", "current_module", "current_submodule", "mistakes_text"],
    template="""You are a language tutor reviewing common mistakes made in this submodule while teaching {learning_language}.

Current Lesson: **Module:** {current_module} → **Submodule:** {current_submodule}

The following mistakes were observed in the past cycles:
{mistakes_text}

**Revision Phase:**  
Provide a concise summary of these mistakes along with revision tips and practice recommendations to help the student improve.
"""
)

# ---------- Compose Output Functions ----------
# Select teaching and testing prompt functions based on user level.
# Assume session is defined and contains "proficiency_level"
session = get_user_session(user_id)  # user_id is assumed to be defined

if session["proficiency_level"].lower() == "beginner":
    teaching_response_func = teaching_template_beginners | llm | StrOutputParser()
    testing_response_func = testing_template_beginners | llm | StrOutputParser()
else:
    teaching_response_func = teaching_template | llm | StrOutputParser()
    testing_response_func = testing_template | llm | StrOutputParser()

evaluation_response_func = evaluation_template | llm | StrOutputParser()
mistake_summary_response_func = mistake_summary_template | llm | StrOutputParser()



# ---------- Initialize Counters ----------
cycle_count = 0
correct_answer_count = 0

# ---------- Conversation Flow Management ----------
phase = "teaching"

print("Chatbot is ready! Type 'exit' at any point to end the conversation.\n")

while True:
    # Load and format chat history
    history_messages = memory.load_memory_variables({}).get("chat_history", [])
    formatted_history = "\n".join(
        f"{'User' if isinstance(msg, HumanMessage) else 'Tutor'}: {msg.content}"
        for msg in history_messages
    ) if history_messages else "No previous conversation."
    
    summarized_history = summarize_chat_history(formatted_history)

    if phase == "teaching":
        teaching_output = teaching_response_func.invoke({
            "learning_language": session["learning_language"],
            "current_module": session["current_module"],
            "current_submodule": session["current_submodule"],
            "chat_history": summarized_history
        })
        print("Tutor (Teaching):")
        print(teaching_output)
        phase = "testing"

    elif phase == "testing":
        testing_output = testing_response_func.invoke({
            "learning_language": session["learning_language"],
            "current_module": session["current_module"],
            "current_submodule": session["current_submodule"],
            "chat_history": summarized_history
        })
        print("\nTutor (Testing):")
        print(testing_output)
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            break
        phase = "evaluation"

    elif phase == "evaluation":
        # Save conversation to get conversation_id (assume save_conversation returns id)
        conversation_id = save_conversation(user_id, session["session_id"], user_input, "")  # temporary placeholder
        
        evaluation_output = evaluation_response_func.invoke({
            "learning_language": session["learning_language"],
            "current_module": session["current_module"],
            "current_submodule": session["current_submodule"],
            "user_response": user_input,
            "chat_history": summarized_history
        })
        print("\nTutor (Evaluation):")
        print(evaluation_output)
        
        # Update correct answer counter
        if "Great job" in evaluation_output:
            correct_answer_count += 1
        else:
            correct_answer_count = max(correct_answer_count - 1, 0)
        
        # Parse evaluation_output for mistakes and store in DB.
        for line in evaluation_output.split("\n"):
            if "→" in line:
                try:
                    parts = line.split("→")
                    if len(parts) < 2:
                        continue
                    left_side = parts[0]
                    if ":" not in left_side:
                        continue
                    error_type = left_side.split(":")[0].strip()
                    incorrect = left_side.split(":")[1].strip()
                    correct_ans = parts[1].strip()
                    if incorrect and correct_ans:
                        save_mistake(session["session_id"], conversation_id, error_type, incorrect, correct_ans, 
                                     module=session["current_module"], submodule=session["current_submodule"])
                        print(f"Logged mistake: {error_type} - {incorrect} → {correct_ans}")
                except Exception as e:
                    print("Error parsing mistake line:", e)
        
        # Save conversation context
        memory.save_context({"user_response": user_input}, {"output": evaluation_output})
        
        cycle_count += 1
        
        # After 5 cycles, present revision summary if mistakes exist
        if cycle_count >= 5:
            conn = sqlite3.connect("chatbot.db")
            cursor = conn.cursor()
            cursor.execute("""
                SELECT error_type, incorrect_response, correct_response 
                FROM mistakes 
                WHERE session_id = ? AND module = ? AND submodule = ?
            """, (session["session_id"], session["current_module"], session["current_submodule"]))
            mistakes_rows = cursor.fetchall()
            conn.close()
            
            if mistakes_rows:
                mistakes_text = "\n".join(
                    f"{row[0]} - {row[1]} → {row[2]}" for row in mistakes_rows
                )
                revision_output = mistake_summary_response_func.invoke({
                    "learning_language": session["learning_language"],
                    "current_module": session["current_module"],
                    "current_submodule": session["current_submodule"],
                    "mistakes_text": mistakes_text
                })
                print("\nTutor (Revision on Mistakes):")
                print(revision_output)
            else:
                print("\nNo mistakes detected in these 5 cycles. Great job!")
            
            # Update progress (advance submodule/module) and reset counters.
            session, cycle_count, correct_answer_count = update_progress(session, cycle_count, correct_answer_count)
        
        phase = "teaching"
        print("\nMoving to the next lesson...\n")


Chatbot is ready! Type 'exit' at any point to end the conversation.

Tutor (Teaching):
**Module 1: Submodule 3 - Introducing Basic Vocabulary**

In this submodule, we will be learning some basic vocabulary words in Hindi to help you start building your language skills. 

**Key Points to Remember:**

1. **Pronunciation:** Hindi is a phonetic language, which means words are pronounced as they are spelled. Pay attention to the sound of each letter to correctly pronounce words.

2. **Word Structure:** Hindi words are typically made up of consonants and vowels. Vowels are pronounced as independent sounds and can also be combined with consonants to form syllables.

3. **Practice:** Practice speaking out loud and writing down the words to help reinforce your learning. Repetition is key to mastering new vocabulary.

Let's get started with some basic words in Hindi:

1. **Namaste** (नमस्ते) - Hello/Greetings
   - Pronunciation: nah-muh-stay

2. **Shukriya** (शुक्रिया) - Thank you
   - Pronuncia